## Requirements
- The XRADIO images are stored xr.Dataset that remains compatible with Xarray
    - Xarray API should work with structure (to_zarr, open_dataset, open_zarr)
        - No numpy data structures in attributes (otherwise serialization fails). Use list and standard numerical types.
        - No other xarray data structures in attributes (look into storing history as just a dict).
    - Allow multiple images and image types as long as they share coordinates.
- Make use of the same convention as Measurement Set Module/Schema
    - `read_image` renamed to `open_image`
    - All measures should be formatted as Xarray-compatible dicts. For example:
    ```Python
        reference_direction = {'attrs': {'frame': 'fk5',
                                'type': 'sky_coord',
                                'units': 'rad',
                                'equinox': 'j2000.0'},
            'data': [3.5392577860590637, 0.5324852164475642],
            'dims': "sky_dir_label",
            'coords': {"sky_dir_label":
                            {"data":["ra","dec"]
                            "dims":"sky_dir_label"}
                    }
        }
    ```
    - Try reusing the coordinate schemas from the measurement set schema (for example, frequency coordinate).
    - Add frame to location called native_projection for native_pole_direction (latpole and lonpole in fits). latpole and lonpole is combined in a single measure called native_pole_direction.
    - Renamed BEAM to BEAM_FIT_PARAMS and beam_param to beam_param_label.
    - Add types to all data variables and dataset.

## Image Types
    sky
    flag
    point_spread_function
    primary_beam
    mask
    beam_fit_params_sky
    beam_fit_params_point_spread_function
    visibility_normalization
    description:
    date:

    visibility*
    visibility_normalization*
    uv_sampling*
    uv_sampling_normalization*
    aperture*
    aperture_normalization*

* UV plain. Only used internally and for debugging.
    
## Questions:
- What is the use case for multiple rest_frequencies (this only seems to be a thing for FITS images)?
- If the addition of datagroups, can we depricate active_mask?
- How to handle the different types of masks:
    - External Mask added as just another image.
    - Internal Masks are currently added by appending the image they come from. For example, MASK_RESIDUAL, MASK_SKY.

## Issues
- Visibility normalization is not loaded correctly (sum_weight). Adds unnecessary l and m coordinates and tiles. 

## Measures example

In [ ]:
import xarray as xr 

reference_direction = {'attrs': {'frame': 'fk5',
                        'type': 'sky_coord',
                        'units': 'rad',
                        'equinox': 'j2000.0'},
    'data': [3.5392577860590637, 0.5324852164475642],
    'dims': "sky_dir_label",
    'coords': {"sky_dir_label":
                    {"data":["ra","dec"],
                     "dims":"sky_dir_label"}
            }
}

dir_xda = xr.DataArray.from_dict(reference_direction)
dir_xda

## Download Data

In [ ]:
import toolviper

# toolviper.utils.data.update()
# Download the test data
# toolviper.utils.data.download("3c286_Band6_5chans_lsrk_robust_0.5_niter_99_fits")
# toolviper.utils.data.download("3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa")

toolviper.utils.data.download("3c286_Band6_5chans_lsrk_robust_0.5_niter_99.fits")
toolviper.utils.data.download("3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa.ms") #Not an ms. Mistake in download metadata. CASA images.

casa_dir = "./3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa/"
fits_dir = "./3c286_Band6_5chans_lsrk_robust_0.5_niter_99_fits/"

import zipfile
from pathlib import Path

zip_files = [
    "./3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa.zip",
    "./3c286_Band6_5chans_lsrk_robust_0.5_niter_99_fits.zip",
]

from pathlib import Path
import zipfile

from pathlib import Path
import zipfile

def unzip_if_present(zip_paths, output_dir=None):
    """
    Unzip ZIP files if they exist.

    If output_dir is given, extract directly into output_dir (no extra folder),
    like: unzip file.zip -d output_dir
    """
    for zip_path in map(Path, zip_paths):
        if not zip_path.exists():
            print(f"ZIP file {zip_path} not found, skipping.")
            continue

        extract_dir = Path(output_dir) if output_dir is not None else zip_path.with_suffix("")
        extract_dir.mkdir(parents=True, exist_ok=True)

        print(f"Extracting {zip_path} to {extract_dir}")
        with zipfile.ZipFile(zip_path, "r") as zf:
            zf.extractall(extract_dir)

unzip_if_present(zip_files,output_dir='.')
import os
print(os.system("ls " + casa_dir)) 
print(os.system("ls " + fits_dir))


Extracting 3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa.zip to .
Extracting 3c286_Band6_5chans_lsrk_robust_0.5_niter_99_fits.zip to .
3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image
3c286_Band6_5chans_lsrk_robust_0.5_niter_99.mask
3c286_Band6_5chans_lsrk_robust_0.5_niter_99.model
3c286_Band6_5chans_lsrk_robust_0.5_niter_99.pb
3c286_Band6_5chans_lsrk_robust_0.5_niter_99.psf
3c286_Band6_5chans_lsrk_robust_0.5_niter_99.residual
3c286_Band6_5chans_lsrk_robust_0.5_niter_99.sumwt
0
3c286_Band6_5chans_lsrk_robust_0.5_niter_99.fits
3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image.fits
3c286_Band6_5chans_lsrk_robust_0.5_niter_99.mask.fits
3c286_Band6_5chans_lsrk_robust_0.5_niter_99.model.fits
3c286_Band6_5chans_lsrk_robust_0.5_niter_99.pb.fits
3c286_Band6_5chans_lsrk_robust_0.5_niter_99.psf.fits
3c286_Band6_5chans_lsrk_robust_0.5_niter_99.residual.fits
3c286_Band6_5chans_lsrk_robust_0.5_niter_99.sumwt.fits
0


# Load and Open a Single Image

In [ ]:
#Opening a single casa or fits image into an xarray Dataset

from toolviper.utils.display import dict_to_html
from IPython.display import HTML, display

from xradio.image import open_image, load_image
import toolviper.utils.logger as logger

casa_dir = "./3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa/"
fits_dir = "./3c286_Band6_5chans_lsrk_robust_0.5_niter_99_fits/"

img_xds = open_image(
        fits_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image.fits",
) #Opening a FITS image 

img_xds = open_image(
        casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image",
)#Opening a CASA image 

img_xds

In [ ]:
display(HTML(dict_to_html(img_xds.attrs)))

In [ ]:
display(HTML(dict_to_html(img_xds.SKY.attrs)))

In [ ]:
#Loading a single casa or fits image into an xarray Dataset

from toolviper.utils.display import dict_to_html
from IPython.display import HTML, display

from xradio.image import open_image, load_image
import toolviper.utils.logger as logger


img_xds = load_image(
        casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image",
)

img_xds

## Loading Mutiple CASA and FITS Images

- VISIBILITY_NORMALIZATION is incorrectly loaded.
- Can mix FITS and CASA images (still have to share coordinates and have the same Dataset level attributes). 

In [ ]:
from xradio.image import open_image

casa_dir = "./3c286_Band6_5chans_lsrk_robust_0.5_niter_99_casa/"
fits_dir = "./3c286_Band6_5chans_lsrk_robust_0.5_niter_99_fits/"

img_xds = open_image(    
    {"sky_deconvolved": casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image",
    "sky_dirty": casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image",
    "sky_model": fits_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.model.fits",
    "sky_residual": casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.residual",
    "mask": casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.mask",
    "primary_beam": fits_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.pb.fits",
    "point_spread_function": casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.psf",
    "visibility_normalization": casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.sumwt"})

img_xds

In [ ]:
from toolviper.utils.display import dict_to_html
from IPython.display import HTML, display

display(HTML(dict_to_html(img_xds.attrs)))

## Writing to Zarr

In [ ]:
import xarray as xr
if "VISIBILITY_NORMALIZATION" in img_xds.data_vars:
    del img_xds["VISIBILITY_NORMALIZATION"] # remove incorrectly loaded dataarray
img_xds.to_zarr("test_zarr_write.zarr", mode="w")
img_write_test_open = xr.open_zarr("test_zarr_write.zarr")
img_write_test_open

## Writing to CASA

- Need to implement a better convention for file extensions.

In [ ]:
from xradio.image import open_image, write_image
write_image(img_xds, "test_casa_write", overwrite=True)
img_xds_written = open_image("test_casa_write.sky")
img_xds_written

## Create an empty image

In [ ]:
from xradio.image import make_empty_sky_image

phase_center = img_xds.attrs['coordinate_system_info']['reference_direction']['data']
image_shape = img_xds.SKY_DECONVOLVED.shape
cell_size = [(img_xds.l[1]-img_xds.l[0]).values.item(), (img_xds.m[1]-img_xds.m[0]).values.item()]
frequency_coords = img_xds.frequency.values.tolist()
pol_coords = img_xds.polarization.values.tolist()
time_coords = img_xds.time.values.tolist()

empty_img_xds = make_empty_sky_image(phase_center=phase_center,
    image_size=image_shape[-2:],
    cell_size=cell_size,
    frequency_coords=frequency_coords,
    pol_coords=pol_coords,
    time_coords=time_coords,
    direction_reference = "fK5",
    projection = "SIN",
    spectral_reference = "lsrk",
    do_sky_coords = True,
)

empty_img_xds

## Image Accessor

In [ ]:
#Loading a single casa or fits image into an xarray Dataset
from xradio.image import open_image


img_xds = open_image(
        casa_dir + "3c286_Band6_5chans_lsrk_robust_0.5_niter_99.image",
)

img_xds

In [ ]:
#Add a new data group with improved image

img_xds["SKY_IMPROVED"] = img_xds["SKY"] * 1.1  # Example improvement: scale the image data


img_xds.xr_img.add_data_group(
        new_data_group_name = "improved",
        new_data_group = {
            "sky": "SKY_IMPROVED",
            "date": "2024-06-10",
            "description": "This data group contains improved image.",
        },
        data_group_dv_shared_with="base"
)

img_xds

In [ ]:
display(HTML(dict_to_html(img_xds.attrs["data_groups"])))

In [ ]:
#Select the improved data group

img_improved_xds = img_xds.xr_img.sel(data_group_name="improved")
img_improved_xds

In [ ]:
img_xds

In [ ]:
img_xds.xr_img.get_reference_pixel_indices()

In [ ]:
img_xds.xr_img.get_lm_cell_size()

In [ ]:
img_xds = img_xds.xr_img.add_uv_coordinates()
img_xds

In [ ]:
img_xds.xr_img.get_uv_in_lambda(frequency=2.399476e+11)

In [ ]:
img_xds.xr_img.get_reference_pixel_indices()

## ?CASA Image Analysis Functions Mapping to RADPS?

- XRADIO Accessors
    - imhead (summary) — summarize and manipulate the “header” information in a CASA image

- XRADIO provided
    - importfits — import a FITS image into a CASA image format table
    - exportfits — write out an image in FITS format

- Use native Xarray
    - imsubimage — Create a (sub)image from a region of the image
    - immath — perform mathematical operations on or between images
    - imval — extract the data and mask values from a pixel or region of an image
    - imtrans — reorder the axes of an image or cube
    - imcollapse — collapse image along one or more axes by aggregating pixel values along that axis
    - makemask — image mask handling

- AstroVIPER
    - xds in, results dict
        - imstat — calculate statistics on an image or part of an image
        - imfit — image plane Gaussian component fitting #component dataset?
        - specfit — fit 1-dimensional Gaussians, polynomial, and/or Lorentzians models to an image or image region
        - specflux — Report details of an image spectrum.
        - spxfit — Calculation of Spectral Indices and higher order polynomials

    - xds in, xds out (same coordinates)
        - imcontsub — perform continuum subtraction on a spectral-line image cube
        - immoments — compute the moments of an image cube
        - specsmooth — 1-dimensional smooth images in the spectral and angular directions
        - imsmooth — 2-dimensional smooth images in the spectral and angular directions
        - rmfit — Calculation of rotation measures

    - xds in, xds out (coordinates changed, new dataset)
        - imregrid — regrid an image onto the coordinate system of another image
        - imreframe — change the frame in which the image reports its spectral values
        - imrebin — rebin an image
        - impv — generate a position-velocity diagram along a slit

    - plotting
        - plotprofilemap — Plot spectra at their position


- Miscellaneous
    - slsearch — query a subset of the Splatalogue spectral line catalog
    - splattotable — convert a file exported from Splatalogue to a CASA table


```
def image_statistics(image_name: str, mask_name: str): #imstat
    stats_dict = {}
    return stats_dict


#User can specify input and output image selections in two ways:
input_image_sel= {"sky":"SKY","mask":"MASK"}
output_image_sel= {"sky":"SKY_SMOOTHED","mask":"MASK"}
or
input_image_sel= {"data_group_name":"base"}
output_image_sel= {"data_group_name":"smoothed"}

def image_smooth(img_xds, input_image_sel, output_image_sel): #imsmooth
    if "data_group_name" in input_image_sel:
        data_group = img_xds.attr["data_groups"][input_image_sel["data_group_name"]]
    else:
        data_group = input_image_sel


    return img_xds
```